In [ ]:
"""
서울시 상권분석서비스 - 길단위인구(행정동) API 호출
- OA-22178
- 기간: 2022년 1분기 ~ 2025년 4분기
- 서비스명: VwsmAdstrdFlorPopltnW
+ EDA (탐색적 데이터 분석) 포함 - 최종 버전
+ 이상치 상세 확인 + CSV 저장
"""

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# ============================================================
# Jupyter 출력 설정 (잘림 방지)
# ============================================================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

# ============================================================
# 설정
# ============================================================
API_KEY = "455963584d68666f32367441675145"
SERVICE_NAME = "VwsmAdstrdFlorPopltnW"
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"

def generate_quarter_codes(start_year=2022, end_year=2025):
    codes = []
    for year in range(start_year, end_year + 1):
        for quarter in range(1, 5):
            codes.append(f"{year}{quarter}")
    return codes

QUARTER_CODES = generate_quarter_codes(2022, 2025)
print(f"수집 대상 분기: {QUARTER_CODES}")
print(f"총 {len(QUARTER_CODES)}개 분기")
sys.stdout.flush()


# ============================================================
# API 호출 함수 (안정화 버전)
# ============================================================
def fetch_all_data():
    """전체 데이터 수집 - Jupyter 안정화 버전"""
    all_data = []
    start = 1
    batch_size = 1000
    max_retries = 3
    max_iterations = 500
    iteration = 0
    
    print(f"\n{'='*60}")
    print("서울시 상권분석 API - 길단위인구(행정동) 데이터 수집 시작")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    while iteration < max_iterations:
        iteration += 1
        end = start + batch_size - 1
        url = f"{BASE_URL}/{start}/{end}"
        
        for retry in range(max_retries):
            try:
                response = requests.get(url, timeout=60)
                response.raise_for_status()
                result = response.json()
                
                if SERVICE_NAME not in result:
                    if 'RESULT' in result:
                        error_msg = result.get('RESULT', {})
                        print(f"API 에러: {error_msg}")
                        if 'INFO-200' in str(error_msg):
                            print("데이터 수집 완료 (더 이상 데이터 없음)")
                            return all_data
                    return all_data
                
                code = result[SERVICE_NAME]['RESULT']['CODE']
                if code != 'INFO-000':
                    print(f"API 응답 코드: {code}")
                    if code == 'INFO-200':
                        return all_data
                    break
                
                rows = result[SERVICE_NAME].get('row', [])
                if not rows:
                    print("더 이상 데이터가 없습니다.")
                    return all_data
                
                all_data.extend(rows)
                total_count = result[SERVICE_NAME]['list_total_count']
                
                print(f"수집 중: {start:,}~{end:,} / 전체 {total_count:,}건 (현재 {len(all_data):,}건)")
                sys.stdout.flush()
                
                if len(all_data) >= total_count:
                    print(f"\n✅ 전체 데이터 수집 완료: {len(all_data):,}건")
                    return all_data
                
                start += batch_size
                time.sleep(0.3)
                break
                
            except requests.exceptions.Timeout:
                print(f"⏰ 타임아웃 (시도 {retry+1}/{max_retries}): {start}~{end}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 에러 (시도 {retry+1}/{max_retries}): {e}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except Exception as e:
                print(f"❌ 처리 에러: {e}")
                sys.stdout.flush()
                return all_data
        
        else:
            print(f"❌ {start}~{end} 구간 수집 실패, 다음으로 진행")
            start += batch_size
            continue
    
    print(f"⚠️ 최대 반복 횟수 도달 ({max_iterations}회)")
    return all_data


def filter_by_quarter(data, quarter_codes):
    """기준년분기코드로 필터링"""
    df = pd.DataFrame(data)
    print(f"\n전체 수집 데이터: {len(df):,}건")
    
    df['STDR_YYQU_CD'] = df['STDR_YYQU_CD'].astype(str)
    df_filtered = df[df['STDR_YYQU_CD'].isin(quarter_codes)]
    print(f"2022~2025년 데이터: {len(df_filtered):,}건")
    
    return df_filtered


# ============================================================
# 컬럼명 한글 변환
# ============================================================
COLUMN_MAPPING = {
    'STDR_YYQU_CD': '기준_년분기_코드',
    'ADSTRD_CD': '행정동_코드',
    'ADSTRD_CD_NM': '행정동_코드_명',
    'TOT_FLPOP_CO': '총_유동인구_수',
    'ML_FLPOP_CO': '남성_유동인구_수',
    'FML_FLPOP_CO': '여성_유동인구_수',
    'AGRDE_10_FLPOP_CO': '연령대_10_유동인구_수',
    'AGRDE_20_FLPOP_CO': '연령대_20_유동인구_수',
    'AGRDE_30_FLPOP_CO': '연령대_30_유동인구_수',
    'AGRDE_40_FLPOP_CO': '연령대_40_유동인구_수',
    'AGRDE_50_FLPOP_CO': '연령대_50_유동인구_수',
    'AGRDE_60_ABOVE_FLPOP_CO': '연령대_60_이상_유동인구_수',
    'TMZON_00_06_FLPOP_CO': '시간대_00~06_유동인구_수',
    'TMZON_06_11_FLPOP_CO': '시간대_06~11_유동인구_수',
    'TMZON_11_14_FLPOP_CO': '시간대_11~14_유동인구_수',
    'TMZON_14_17_FLPOP_CO': '시간대_14~17_유동인구_수',
    'TMZON_17_21_FLPOP_CO': '시간대_17~21_유동인구_수',
    'TMZON_21_24_FLPOP_CO': '시간대_21~24_유동인구_수',
    'MON_FLPOP_CO': '월요일_유동인구_수',
    'TUES_FLPOP_CO': '화요일_유동인구_수',
    'WED_FLPOP_CO': '수요일_유동인구_수',
    'THUR_FLPOP_CO': '목요일_유동인구_수',
    'FRI_FLPOP_CO': '금요일_유동인구_수',
    'SAT_FLPOP_CO': '토요일_유동인구_수',
    'SUN_FLPOP_CO': '일요일_유동인구_수',
}


# ============================================================
# EDA 함수 (최종 버전 - 이상치 상세 + CSV 저장)
# ============================================================
def run_eda(df):
    """
    탐색적 데이터 분석 (EDA) - 유동인구
    - 이상치 상세 확인
    - CSV 파일로 저장 (출력 잘림 방지)
    """
    print(f"\n{'='*60}")
    print("📊 EDA (탐색적 데이터 분석) 시작 - 유동인구")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    # =========================================================
    # 1단계: 데이터 훑어보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 1단계: 데이터 훑어보기")
    print(f"{'='*60}")
    
    print("\n[1-1] 데이터 크기")
    print("-" * 40)
    print(f"행 개수: {df.shape[0]:,}개")
    print(f"열 개수: {df.shape[1]}개")
    
    print("\n[1-2] 컬럼 목록")
    print("-" * 40)
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2}. {col}")
    
    print("\n[1-3] 데이터 타입")
    print("-" * 40)
    print(df.dtypes)
    
    print("\n[1-4] 상위 5개 행 미리보기")
    print("-" * 40)
    print(df.head())
    
    print("\n[1-5] 결측치 확인")
    print("-" * 40)
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({'결측치 수': missing, '결측치 비율(%)': missing_pct})
    missing_with_values = missing_df[missing_df['결측치 수'] > 0]
    
    if len(missing_with_values) > 0:
        print(missing_with_values)
        # 결측치 샘플 CSV 저장
        for col in missing_with_values.index[:3]:
            sample = df[df[col].isnull()][['행정동_코드_명', '기준_년분기_코드', col]]
            sample.to_csv(f'eda_결측치_{col}.csv', index=False, encoding='utf-8-sig')
        print(f"\n✅ 결측치 샘플 CSV 저장 완료")
    else:
        print("✅ 결측치 없음!")
    
    sys.stdout.flush()
    
    # =========================================================
    # 2단계: 숫자 요약하기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 2단계: 숫자 요약하기")
    print(f"{'='*60}")
    
    print("\n[2-1] 기초 통계량 (총_유동인구_수)")
    print("-" * 40)
    if '총_유동인구_수' in df.columns:
        stats = df['총_유동인구_수'].describe()
        print(f"개수: {stats['count']:,.0f}개")
        print(f"평균: {stats['mean']:,.0f}명")
        print(f"최소: {stats['min']:,.0f}명")
        print(f"최대: {stats['max']:,.0f}명")
        print(f"중앙값: {stats['50%']:,.0f}명")
        print(f"표준편차: {stats['std']:,.0f}명")
    
    sys.stdout.flush()
    
    # =========================================================
    # 3단계: 그래프로 보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 3단계: 그래프로 보기")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    if '총_유동인구_수' in df.columns:
        axes[0, 0].hist(df['총_유동인구_수'] / 1e6, bins=50, edgecolor='black', alpha=0.7)
        axes[0, 0].set_title('총 유동인구 분포 (히스토그램)', fontsize=12)
        axes[0, 0].set_xlabel('유동인구 (백만명)')
        axes[0, 0].set_ylabel('빈도')
    
    if '총_유동인구_수' in df.columns:
        axes[0, 1].boxplot(df['총_유동인구_수'] / 1e6, vert=True)
        axes[0, 1].set_title('총 유동인구 박스플롯 (이상치 확인)', fontsize=12)
        axes[0, 1].set_ylabel('유동인구 (백만명)')
    
    if '기준_년분기_코드' in df.columns and '총_유동인구_수' in df.columns:
        quarterly_avg = df.groupby('기준_년분기_코드')['총_유동인구_수'].mean() / 1e6
        quarterly_avg.plot(kind='bar', ax=axes[1, 0], color='steelblue', edgecolor='black')
        axes[1, 0].set_title('분기별 평균 유동인구', fontsize=12)
        axes[1, 0].set_xlabel('분기')
        axes[1, 0].set_ylabel('평균 유동인구 (백만명)')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    day_cols = ['월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', 
                '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수']
    day_cols = [col for col in day_cols if col in df.columns]
    if day_cols:
        day_means = df[day_cols].mean() / 1e6
        day_labels = ['월', '화', '수', '목', '금', '토', '일']
        axes[1, 1].bar(day_labels, day_means, color='coral', edgecolor='black')
        axes[1, 1].set_title('요일별 평균 유동인구', fontsize=12)
        axes[1, 1].set_xlabel('요일')
        axes[1, 1].set_ylabel('평균 유동인구 (백만명)')
    
    plt.tight_layout()
    plt.savefig('eda_유동인구_시각화.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ 시각화 저장 완료: eda_유동인구_시각화.png")
    
    sys.stdout.flush()
    
    # =========================================================
    # 4단계: 이상한 점 발견하기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 4단계: 이상한 점 발견하기")
    print(f"{'='*60}")
    
    # ---------------------------------------------------------
    # 4-1. 유동인구가 0인 데이터
    # ---------------------------------------------------------
    print("\n[4-1] 유동인구가 0인 데이터")
    print("-" * 40)
    if '총_유동인구_수' in df.columns:
        zero_pop = df[df['총_유동인구_수'] == 0]
        print(f"유동인구 0 데이터: {len(zero_pop):,}건 ({len(zero_pop)/len(df)*100:.2f}%)")
        
        if len(zero_pop) > 0:
            # CSV 저장
            zero_pop_save = zero_pop[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']].copy()
            zero_pop_save.to_csv('eda_유동인구_0인_데이터.csv', index=False, encoding='utf-8-sig')
            print(f"✅ CSV 저장 완료: eda_유동인구_0인_데이터.csv ({len(zero_pop)}건)")
            
            print("\n📍 유동인구 0이 많은 행정동 TOP 5:")
            zero_by_dong = zero_pop['행정동_코드_명'].value_counts().head(5)
            for dong, cnt in zero_by_dong.items():
                print(f"  - {dong}: {cnt}건")
        else:
            print("✅ 유동인구 0인 데이터 없음!")
    
    # ---------------------------------------------------------
    # 4-2. 유동인구가 음수인 데이터
    # ---------------------------------------------------------
    print("\n[4-2] 유동인구가 음수인 데이터")
    print("-" * 40)
    if '총_유동인구_수' in df.columns:
        negative_pop = df[df['총_유동인구_수'] < 0]
        print(f"유동인구 음수 데이터: {len(negative_pop):,}건")
        
        if len(negative_pop) > 0:
            negative_pop[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']].to_csv(
                'eda_유동인구_음수_데이터.csv', index=False, encoding='utf-8-sig')
            print(f"✅ CSV 저장 완료: eda_유동인구_음수_데이터.csv")
        else:
            print("✅ 음수 유동인구 없음!")
    
    # ---------------------------------------------------------
    # 4-3. 중복 데이터 확인
    # ---------------------------------------------------------
    print("\n[4-3] 중복 데이터 확인")
    print("-" * 40)
    
    dup_all = df.duplicated().sum()
    print(f"전체 행 중복: {dup_all:,}건")
    
    if '행정동_코드' in df.columns and '기준_년분기_코드' in df.columns:
        dup_key = df.duplicated(subset=['행정동_코드', '기준_년분기_코드'], keep=False)
        dup_data = df[dup_key]
        print(f"행정동+분기 기준 중복: {len(dup_data):,}건")
        
        if len(dup_data) > 0:
            dup_data.to_csv('eda_유동인구_중복_데이터.csv', index=False, encoding='utf-8-sig')
            print(f"✅ CSV 저장 완료: eda_유동인구_중복_데이터.csv")
    
    # ---------------------------------------------------------
    # 4-4. 이상치 확인 (IQR 방식) - 상세 버전
    # ---------------------------------------------------------
    print("\n[4-4] 이상치 확인 (총_유동인구_수 - IQR 방식)")
    print("-" * 40)
    
    total_outliers = 0
    outliers_low = pd.DataFrame()
    outliers_high = pd.DataFrame()
    
    if '총_유동인구_수' in df.columns:
        q1 = df['총_유동인구_수'].quantile(0.25)
        q3 = df['총_유동인구_수'].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        
        print("\n📐 IQR 이상치 판별 기준:")
        print(f"  - Q1 (25%): {q1:,.0f}명")
        print(f"  - Q3 (75%): {q3:,.0f}명")
        print(f"  - IQR (Q3-Q1): {iqr:,.0f}명")
        print(f"  - 하한 경계 (Q1 - 1.5*IQR): {lower:,.0f}명")
        print(f"  - 상한 경계 (Q3 + 1.5*IQR): {upper:,.0f}명")
        print(f"\n  → 유동인구 < {lower:,.0f}명 이면 '하한 이상치'")
        print(f"  → 유동인구 > {upper:,.0f}명 이면 '상한 이상치'")
        
        # 하한 이상치
        outliers_low = df[df['총_유동인구_수'] < lower].copy()
        print(f"\n[하한 이상치] {len(outliers_low):,}건")
        
        if len(outliers_low) > 0:
            outliers_low_save = outliers_low[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']].copy()
            outliers_low_save['유동인구(만명)'] = (outliers_low_save['총_유동인구_수'] / 1e4).round(1)
            outliers_low_save = outliers_low_save.sort_values('총_유동인구_수')
            outliers_low_save.to_csv('eda_유동인구_하한이상치.csv', index=False, encoding='utf-8-sig')
            print(f"✅ CSV 저장 완료: eda_유동인구_하한이상치.csv ({len(outliers_low)}건)")
            
            print("\n⚠️ 하한 이상치 TOP 10 (유동인구 가장 적은 순):")
            top10_low = outliers_low.nsmallest(10, '총_유동인구_수')[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']]
            for idx, row in top10_low.iterrows():
                print(f"  - {row['행정동_코드_명']} ({row['기준_년분기_코드']}): {row['총_유동인구_수']:,.0f}명")
            
            print("\n📍 하한 이상치 빈도 TOP 10 (어떤 행정동이 자주 이상치로 잡히나):")
            freq_low = outliers_low.groupby('행정동_코드_명').agg(
                이상치_횟수=('총_유동인구_수', 'count'),
                평균_유동인구=('총_유동인구_수', 'mean')
            ).sort_values('이상치_횟수', ascending=False).head(10)
            freq_low['평균(만명)'] = (freq_low['평균_유동인구'] / 1e4).round(1)
            for dong, row in freq_low.iterrows():
                print(f"  - {dong}: {row['이상치_횟수']}건 (평균 {row['평균(만명)']}만명)")
        
        # 상한 이상치
        outliers_high = df[df['총_유동인구_수'] > upper].copy()
        print(f"\n[상한 이상치] {len(outliers_high):,}건")
        
        if len(outliers_high) > 0:
            outliers_high_save = outliers_high[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']].copy()
            outliers_high_save['유동인구(만명)'] = (outliers_high_save['총_유동인구_수'] / 1e4).round(1)
            outliers_high_save = outliers_high_save.sort_values('총_유동인구_수', ascending=False)
            outliers_high_save.to_csv('eda_유동인구_상한이상치.csv', index=False, encoding='utf-8-sig')
            print(f"✅ CSV 저장 완료: eda_유동인구_상한이상치.csv ({len(outliers_high)}건)")
            
            print("\n⚠️ 상한 이상치 TOP 10 (유동인구 가장 많은 순):")
            top10_high = outliers_high.nlargest(10, '총_유동인구_수')[['행정동_코드_명', '기준_년분기_코드', '총_유동인구_수']]
            for idx, row in top10_high.iterrows():
                print(f"  - {row['행정동_코드_명']} ({row['기준_년분기_코드']}): {row['총_유동인구_수']:,.0f}명 ({row['총_유동인구_수']/1e4:.0f}만명)")
            
            print("\n📍 상한 이상치 빈도 TOP 10 (어떤 행정동이 자주 이상치로 잡히나):")
            freq_high = outliers_high.groupby('행정동_코드_명').agg(
                이상치_횟수=('총_유동인구_수', 'count'),
                평균_유동인구=('총_유동인구_수', 'mean')
            ).sort_values('이상치_횟수', ascending=False).head(10)
            freq_high['평균(만명)'] = (freq_high['평균_유동인구'] / 1e4).round(1)
            for dong, row in freq_high.iterrows():
                print(f"  - {dong}: {row['이상치_횟수']}건 (평균 {row['평균(만명)']}만명)")
        
        total_outliers = len(outliers_low) + len(outliers_high)
        print(f"\n📊 이상치 요약:")
        print(f"  - 하한 이상치: {len(outliers_low):,}건")
        print(f"  - 상한 이상치: {len(outliers_high):,}건")
        print(f"  - 전체 이상치: {total_outliers:,}건 ({total_outliers/len(df)*100:.1f}%)")
    
    # ---------------------------------------------------------
    # 4-5. 분기별 데이터 건수 확인
    # ---------------------------------------------------------
    print("\n[4-5] 분기별 데이터 건수 확인")
    print("-" * 40)
    if '기준_년분기_코드' in df.columns:
        quarter_counts = df.groupby('기준_년분기_코드').size().sort_index()
        print(quarter_counts)
        
        avg_count = quarter_counts.mean()
        print(f"\n분기 평균 데이터 수: {avg_count:.0f}건")
        
        abnormal_quarters = quarter_counts[abs(quarter_counts - avg_count) > avg_count * 0.2]
        if len(abnormal_quarters) > 0:
            print("\n⚠️ 평균 대비 ±20% 이상 차이나는 분기:")
            for q, cnt in abnormal_quarters.items():
                diff_pct = (cnt - avg_count) / avg_count * 100
                print(f"  - {q}: {cnt}건 ({diff_pct:+.1f}%)")
    
    sys.stdout.flush()
    
    # =========================================================
    # EDA 요약
    # =========================================================
    print(f"\n{'='*60}")
    print("📊 EDA 요약 - 유동인구")
    print(f"{'='*60}")
    print(f"총 데이터: {len(df):,}건")
    print(f"행정동 수: {df['행정동_코드_명'].nunique()}개")
    print(f"분기 수: {df['기준_년분기_코드'].nunique()}개")
    
    issues = []
    if len(missing_with_values) > 0:
        issues.append(f"결측치 있음")
    if '총_유동인구_수' in df.columns and len(df[df['총_유동인구_수'] == 0]) > 0:
        issues.append(f"유동인구 0: {len(df[df['총_유동인구_수'] == 0]):,}건")
    if '총_유동인구_수' in df.columns and len(df[df['총_유동인구_수'] < 0]) > 0:
        issues.append(f"유동인구 음수: {len(df[df['총_유동인구_수'] < 0]):,}건")
    if dup_all > 0:
        issues.append(f"중복 행: {dup_all:,}건")
    if '총_유동인구_수' in df.columns:
        issues.append(f"이상치: {total_outliers:,}건")
    
    if issues:
        print(f"\n⚠️ 발견된 이슈:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("\n✅ 특별한 이슈 없음!")
    
    print(f"\n📁 저장된 CSV 파일 목록:")
    print("  - eda_유동인구_하한이상치.csv (유동인구 너무 적은 행정동)")
    print("  - eda_유동인구_상한이상치.csv (유동인구 너무 많은 행정동)")
    if len(zero_pop) > 0:
        print("  - eda_유동인구_0인_데이터.csv")
    if len(negative_pop) > 0:
        print("  - eda_유동인구_음수_데이터.csv")
    if len(dup_data) > 0:
        print("  - eda_유동인구_중복_데이터.csv")
    
    print(f"\n{'='*60}")
    print("📊 EDA 완료!")
    print(f"{'='*60}")
    
    return df


# ============================================================
# 메인 실행
# ============================================================
if __name__ == "__main__":
    print("🚀 프로그램 시작")
    print(f"현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    sys.stdout.flush()
    
    if API_KEY == "YOUR_API_KEY_HERE":
        print("⚠️  API_KEY를 설정해주세요!")
        API_KEY = "sample"
        BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"
    
    raw_data = fetch_all_data()
    
    if raw_data:
        df_result = filter_by_quarter(raw_data, QUARTER_CODES)
        
        if len(df_result) > 0:
            existing_cols = {k: v for k, v in COLUMN_MAPPING.items() if k in df_result.columns}
            df_result = df_result.rename(columns=existing_cols)
            
            numeric_cols = [col for col in df_result.columns if '유동인구' in col]
            for col in numeric_cols:
                df_result[col] = pd.to_numeric(df_result[col], errors='coerce')
            
            output_file = "서울시_길단위인구_행정동_2022_2025.csv"
            df_result.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"\n✅ 저장 완료: {output_file}")
            
            print(f"\n{'='*60}")
            print("수집 결과 요약")
            print(f"{'='*60}")
            print(f"총 데이터 수: {len(df_result):,}건")
            print(f"행정동 수: {df_result['행정동_코드_명'].nunique()}개")
            print(f"분기 수: {df_result['기준_년분기_코드'].nunique()}개")
            
            df_result = run_eda(df_result)
            
        else:
            print("2022~2025년 데이터가 없습니다.")
    else:
        print("데이터 수집 실패")
    
    print(f"\n🏁 프로그램 종료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")